# OC PROJET 4 - CLIENT SEGMENTATION
#### SIMULATION NOTEBOOK
<br></br>
### SOMMAIRE

- <a href="#C1">I. Simulations mensuelles</a>
    - 1. Clustering Janvier 2017
    - 2. Clustering Février 2017
    - 3. Clustering Mars 2017
<br></br>    
- <a href="#C2">II. Simulations trimestrielles</a>
    - 1. Clustering 1er Trimestre 2017
    - 2. Clustering 2eme Trimestre 2017
    - 3. Clustering 3eme Trimestre 2017
    - 4. Clustering 4eme Trimestre 2017

Note sur la convention PEP8 : le code à été écrit en suivant la convention PEP8, 
certaines lignes ne respectent pas les règles de la convention mais comme convenu dans
les guidelines j'ai décider volontairement de les laisser tel quel car sinon cela
casserai le code.

https://peps.python.org/pep-0008/

In [25]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import font_manager as fm, rcParams
import matplotlib.patheffects as path_effects
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import scipy.stats as st
from scipy.stats import pearsonr
from scipy.stats import f_oneway
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import pairwise_distances

In [26]:
# chargement multiple des dataframes
df_customers = pd.read_csv('olist_customers_dataset.csv')
df_orders = pd.read_csv('olist_orders_dataset.csv')
df_order_review = pd.read_csv('olist_order_reviews_dataset.csv')
df_order_items = pd.read_csv('olist_order_items_dataset.csv')
df_order_payments = pd.read_csv('olist_order_payments_dataset.csv')
df_products = pd.read_csv('olist_products_dataset.csv')
df_products_cat = pd.read_csv('product_category_name_translation.csv')
df_sellers = pd.read_csv('olist_sellers_dataset.csv')

In [27]:
# fusion des dataframes vers un dataframe principal
df_merged = pd.merge(df_customers, df_orders, 
how = 'left', on = 'customer_id')
df_merged_a = pd.merge(df_merged, df_order_review, 
how = 'left', on = 'order_id')
df_merged_b = pd.merge(df_merged_a, df_order_items, 
how = 'left', on = 'order_id')
df_merged_c = pd.merge(df_merged_b, df_products, 
how = 'left', on = 'product_id')
df_merged_d = pd.merge(df_merged_c, df_products_cat, 
how = 'left', on = 'product_category_name')
df_merged_e = pd.merge(df_merged_d, df_sellers, 
how = 'left', on = 'seller_id')
df = pd.merge(df_merged_e, df_order_payments, 
how = 'left', on = 'order_id')

In [28]:
# suppression des variables inutiles
df = df.drop(columns = ['order_approved_at', 'order_delivered_carrier_date',
'order_estimated_delivery_date', 'review_id', 'review_answer_timestamp',
'order_item_id', 'payment_sequential', 'product_name_lenght', 
'product_description_lenght', 'product_photos_qty', 'product_weight_g',
'product_length_cm', 'product_height_cm', 'product_width_cm',
'product_category_name', 'shipping_limit_date', 'customer_zip_code_prefix',
'seller_zip_code_prefix', 'review_creation_date'])

df = df.drop(columns = ['customer_id', 'order_id', 'order_status',
'product_id', 'seller_id', 'review_comment_title', 'review_comment_message'])

# suppression des valeurs manquantes
df = df.dropna()

# Suppression des doublons
df.drop_duplicates(inplace = True)

In [29]:
# création de deux nouvelles variables en faisant
# une conversion vers le format datetime
df['order_purchase_datetime'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_datetime'] = pd.to_datetime(df['order_delivered_customer_date'])

# supression des anciennes colonnes du mauvais dtype
df = df.drop(columns = ['order_purchase_timestamp', 
'order_delivered_customer_date'])

In [30]:
# Convertir la colonne 'order_purchase_datetime' en format de date
df['order_purchase_datetime'] = pd.to_datetime(df['order_purchase_datetime'])

# Filtrer les échantillons pour chaque critère
df_jan_2017 = df[(df['order_purchase_datetime'].dt.year == 2017) 
& (df['order_purchase_datetime'].dt.month == 1)]
df_fev_2017 = df[(df['order_purchase_datetime'].dt.year == 2017) 
& (df['order_purchase_datetime'].dt.month == 2)]
df_mar_2017 = df[(df['order_purchase_datetime'].dt.year == 2017) 
& (df['order_purchase_datetime'].dt.month <= 3)]

# Filtrer les échantillons pour chaque critère
df_t1_2017 = df[(df['order_purchase_datetime'].dt.year == 2017) 
& (df['order_purchase_datetime'].dt.month <= 3)]
df_t2_2017 = df[(df['order_purchase_datetime'].dt.year == 2017) 
& (df['order_purchase_datetime'].dt.month > 3)
& (df['order_purchase_datetime'].dt.month <= 6)]
df_t3_2017 = df[(df['order_purchase_datetime'].dt.year == 2017) 
& (df['order_purchase_datetime'].dt.month > 6)
& (df['order_purchase_datetime'].dt.month <= 9)]
df_t4_2017 = df[(df['order_purchase_datetime'].dt.year == 2017) 
& (df['order_purchase_datetime'].dt.month > 9)
& (df['order_purchase_datetime'].dt.month <= 12)]

<font size="5">Clustering sur le dataframe entier pour comparer ensuite</font>

In [35]:
# Chargement des données
data = pd.read_csv('olist_master.csv')

# Sélection des colonnes pertinentes pour la segmentation
selected_columns = ['total_amount_client', 'review_score', 
'days_since_last_purchase', 'payment_installments']

data = data[selected_columns]

X = data[selected_columns]

# Prétraitement des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Application de l'algorithme K-means
kmeans = KMeans(n_clusters = 4, random_state = 42)
kmeans.fit(X_scaled)

# Ajout des étiquettes de clusters au dataframe d'origine
data['cluster'] = kmeans.labels_

# Obtenir les labels prédits
kmeans_labels = kmeans.labels_

# Analyse des clusters
cluster_counts = data['cluster'].value_counts()
print("Nombre de clients par cluster:")
print(cluster_counts)

Nombre de clients par cluster:
0    41197
2    30631
3    15949
1    13504
Name: cluster, dtype: int64


# <a name="C1">I. Simulations mensuelles</a>

<font size="5">1. Janvier 2017</font>

In [31]:
#feature engineering

# Créer la variable du nombre de jours depuis le dernier achat
limit_date = pd.to_datetime('2018-09-01')
last_purchase = df_jan_2017.groupby('customer_unique_id')['order_purchase_datetime'].max()
df_jan_2017['days_since_last_purchase'] = (limit_date 
- df_jan_2017['customer_unique_id'].map(last_purchase)).dt.days

# remplacer les valeurs manquantes avec 
# le temps écoulé depuis la dernière observation
df_jan_2017['days_since_last_purchase'].fillna((limit_date 
- df_jan_2017['order_purchase_datetime']).dt.days, inplace = True)

# Créer la variable du montant total des achats effectués par le client
total_purchases = df_jan_2017.groupby('customer_unique_id')['payment_value'].sum()
df_jan_2017['total_amount_client'] = df_jan_2017['customer_unique_id'].map(total_purchases)

C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\2357724978.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_2017['days_since_last_purchase'] = (limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\2357724978.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_2017['days_since_last_purchase'].fillna((limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\2357724978.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [32]:
df_jan_2017.shape

(806, 16)

In [33]:
df_jan_2017

,customer_unique_id,customer_city,customer_state,review_score,price,freight_value,product_category_name_english,seller_city,seller_state,payment_type,payment_installments,payment_value,order_purchase_datetime,order_delivered_datetime,days_since_last_purchase,total_amount_client
55,e021e698833bdeb89dfef3acb2e91f37,jaragua do sul,SC,5.0,223.90,15.74,consoles_games,cascavel,PR,credit_card,4.0,239.64,2017-01-26 11:14:58,2017-02-09 12:15:20,582,239.64
234,c68949b701afe5343ce845dd492772e8,santiago,RS,5.0,22.90,16.05,furniture_decor,ibitinga,SP,credit_card,1.0,38.95,2017-01-31 19:07:34,2017-02-14 11:12:29,577,38.95
428,6b7e1090d106aaee31d5ed12d2bbc2af,sao paulo,SP,1.0,69.90,18.00,sports_leisure,ribeirao preto,SP,credit_card,1.0,87.90,2017-01-29 13:42:10,2017-05-12 13:43:02,579,87.90
849,fc19fe9ac733e36ff8227c29435c2030,jales,SP,5.0,49.90,10.96,fashion_bags_accessories,sao paulo,SP,voucher,1.0,33.05,2017-01-25 00:46:17,2017-01-31 17:43:01,583,60.86
850,fc19fe9ac733e36ff8227c29435c2030,jales,SP,5.0,49.90,10.96,fashion_bags_accessories,sao paulo,SP,credit_card,1.0,27.81,2017-01-25 00:46:17,2017-01-31 17:43:01,583,60.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118773,ed05a1bbf9f8816af23fcc68084bb87d,embu,SP,5.0,99.90,13.28,furniture_decor,ibitinga,SP,credit_card,1.0,452.72,2017-01-16 13:17:46,2017-01-25 08:43:39,592,509.58
118795,b4a76bacd02c4823afd23c34e8ea92e4,blumenau,SC,4.0,79.99,18.00,bed_bath_table,ibitinga,SP,credit_card,3.0,97.99,2017-01-30 20:26:28,2017-02-13 17:19:34,578,97.99
118827,408aee96c75632a92e5079eee61da399,rio de janeiro,RJ,5.0,349.99,26.90,baby,rio de janeiro,RJ,credit_card,7.0,376.89,2017-01-31 14:38:27,2017-02-07 16:31:33,577,376.89
118845,f7be9bec658c62ab6240b44cd26c0b84,curitiba,PR,5.0,9.90,8.72,furniture_bedroom,curitiba,PR,boleto,1.0,18.62,2017-01-05 13:34:35,2017-01-17 15:52:28,603,38.24


In [34]:
# Sélection des colonnes pertinentes pour la segmentation
selected_columns = ['total_amount_client', 'review_score', 
'days_since_last_purchase', 'payment_installments']

df_jan_2017 = df_jan_2017[selected_columns]

X = df_jan_2017[selected_columns]

# Prétraitement des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Application de l'algorithme K-means
kmeans_jan_2017 = KMeans(n_clusters = 4, random_state = 42)
kmeans_jan_2017.fit(X_scaled)

# Obtenir les labels prédits
kmeans_jan_2017_labels = kmeans_jan_2017.labels_

# Ajout des étiquettes de clusters au dataframe d'origine
df_jan_2017['cluster'] = kmeans_jan_2017.labels_

# Analyse des clusters
cluster_counts = df_jan_2017['cluster'].value_counts()
print("Nombre de clients par cluster:")
print(cluster_counts)

# drop de la colonne pour préserver la forme des données
# en vue d'un futur ARI
df_jan_2017 = df_jan_2017.drop(['cluster'], axis = 1)

Nombre de clients par cluster:
0    354
2    216
3    126
1    110
Name: cluster, dtype: int64


C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\622672286.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_2017['cluster'] = kmeans_jan_2017.labels_


In [36]:
# Calculer la matrice de similarité entre les deux ensembles de clusters
similarity_matrix = pairwise_distances(kmeans_jan_2017_labels.reshape(-1, 1), 
kmeans_labels.reshape(-1, 1), metric='hamming')

# Calculer le Clustering Stability Index
stability_index = np.mean(similarity_matrix)

# Afficher le Clustering Stability Index
print("Clustering Stability Index:", stability_index)

Clustering Stability Index: 0.6974850429031403


In [37]:
adjusted_rand_score(kmeans_jan_2017.predict(df_jan_2017), 
kmeans.predict(df_jan_2017))

C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


0.8998218708750507

<font size="5">2. Février 2017</font>

In [41]:
#feature engineering

# Créer la variable du nombre de jours depuis le dernier achat
limit_date = pd.to_datetime('2018-09-01')
last_purchase = df_fev_2017.groupby('customer_unique_id')['order_purchase_datetime'].max()
df_fev_2017['days_since_last_purchase'] = (limit_date 
- df_fev_2017['customer_unique_id'].map(last_purchase)).dt.days

# remplacer les valeurs manquantes avec 
# le temps écoulé depuis la dernière observation
df_fev_2017['days_since_last_purchase'].fillna((limit_date 
- df_fev_2017['order_purchase_datetime']).dt.days, inplace = True)

# Créer la variable du montant total des achats effectués par le client
total_purchases = df_fev_2017.groupby('customer_unique_id')['payment_value'].sum()
df_fev_2017['total_amount_client'] = df_fev_2017['customer_unique_id'].map(total_purchases)

C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\1172465975.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fev_2017['days_since_last_purchase'] = (limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\1172465975.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fev_2017['days_since_last_purchase'].fillna((limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\1172465975.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [42]:
df_fev_2017.shape

(1715, 16)

In [43]:
df_fev_2017

,customer_unique_id,customer_city,customer_state,review_score,price,freight_value,product_category_name_english,seller_city,seller_state,payment_type,payment_installments,payment_value,order_purchase_datetime,order_delivered_datetime,days_since_last_purchase,total_amount_client
132,4bc7250d4b42b3374eb6f41b12429592,sao paulo,SP,5.0,239.0,12.28,watches_gifts,sumare,SP,credit_card,8.0,251.28,2017-02-06 16:49:19,2017-02-11 12:38:56,571,251.28
140,24a5c2b24a4467c37e2f54c1b311de93,marialva,PR,1.0,184.9,15.46,housewares,salto,SP,boleto,1.0,200.36,2017-02-08 16:06:47,2017-02-21 17:08:42,569,200.36
151,96b1c916f245f2462efcde3fce04bc3e,belo horizonte,MG,5.0,168.0,17.61,baby,curitiba,PR,credit_card,3.0,185.61,2017-02-13 07:34:18,2017-02-25 08:37:56,564,185.61
166,1d4626b197f66aa6129c135ab09f1a7b,brasilia,DF,4.0,635.9,43.53,small_appliances,sao paulo,SP,boleto,1.0,679.43,2017-02-13 10:20:42,2017-02-23 11:11:58,564,679.43
186,de53649541ad2006de823720ad65ca77,belo horizonte,MG,3.0,157.0,18.85,garden_tools,sao paulo,SP,credit_card,3.0,175.85,2017-02-09 10:40:05,2017-02-14 13:08:44,568,175.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118747,70f0fca2435f0fa72fe85699630791fc,cafelandia,PR,3.0,48.0,24.84,housewares,mucambo,CE,credit_card,8.0,145.68,2017-02-09 11:01:14,2017-03-02 11:24:05,568,617.10
118831,60dce986762313a6e0b5d1ff0ecde379,sao paulo,SP,5.0,69.9,11.10,furniture_decor,santa barbara d'oeste,SP,credit_card,1.0,81.00,2017-02-04 15:05:16,2017-02-07 10:05:08,573,81.00
118862,203443313490c84f08aadead1dbde82f,campinas,SP,5.0,51.9,11.75,computers_accessories,salto,SP,credit_card,3.0,63.65,2017-02-11 13:39:30,2017-02-20 11:22:10,566,63.65
119002,adc5def00980baefc876eb8023fb053d,sao paulo,SP,5.0,209.9,18.21,computers_accessories,maringa,PR,credit_card,6.0,228.11,2017-02-16 19:09:51,2017-02-21 13:37:49,561,228.11


In [44]:
# Sélection des colonnes pertinentes pour la segmentation
selected_columns = ['total_amount_client', 'review_score', 
'days_since_last_purchase', 'payment_installments']

df_fev_2017 = df_fev_2017[selected_columns]

X = df_fev_2017[selected_columns]

# Prétraitement des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Application de l'algorithme K-means
kmeans_fev_2017 = KMeans(n_clusters = 4, random_state = 42)
kmeans_fev_2017.fit(X_scaled)

# Obtenir les labels prédits
kmeans_fev_2017_labels = kmeans_fev_2017.labels_

# Ajout des étiquettes de clusters au dataframe d'origine
df_fev_2017['cluster'] = kmeans_fev_2017.labels_

# Analyse des clusters
cluster_counts = df_fev_2017['cluster'].value_counts()
print("Nombre de clients par cluster:")
print(cluster_counts)

# drop de la colonne pour préserver la forme des données
# en vue d'un futur ARI
df_fev_2017 = df_fev_2017.drop(['cluster'], axis = 1)

Nombre de clients par cluster:
3    689
0    514
1    256
2    256
Name: cluster, dtype: int64


C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\3522394791.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fev_2017['cluster'] = kmeans_fev_2017.labels_


In [45]:
# Calculer la matrice de similarité entre les deux ensembles de clusters
similarity_matrix = pairwise_distances(kmeans_fev_2017_labels.reshape(-1, 1), 
kmeans_labels.reshape(-1, 1), metric='hamming')

# Calculer le Clustering Stability Index
stability_index = np.mean(similarity_matrix)

# Afficher le Clustering Stability Index
print("Clustering Stability Index:", stability_index)

Clustering Stability Index: 0.7497786359648356


In [47]:
adjusted_rand_score(kmeans_fev_2017.predict(df_fev_2017), 
kmeans.predict(df_fev_2017))

C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


0.8661946282706151

<font size="5">3. Mars 2017</font>

In [48]:
#feature engineering

# Créer la variable du nombre de jours depuis le dernier achat
limit_date = pd.to_datetime('2018-09-01')
last_purchase = df_mar_2017.groupby('customer_unique_id')['order_purchase_datetime'].max()
df_mar_2017['days_since_last_purchase'] = (limit_date 
- df_mar_2017['customer_unique_id'].map(last_purchase)).dt.days

# remplacer les valeurs manquantes avec 
# le temps écoulé depuis la dernière observation
df_mar_2017['days_since_last_purchase'].fillna((limit_date 
- df_mar_2017['order_purchase_datetime']).dt.days, inplace = True)

# Créer la variable du montant total des achats effectués par le client
total_purchases = df_mar_2017.groupby('customer_unique_id')['payment_value'].sum()
df_mar_2017['total_amount_client'] = df_mar_2017['customer_unique_id'].map(total_purchases)

C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\3146139932.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mar_2017['days_since_last_purchase'] = (limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\3146139932.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mar_2017['days_since_last_purchase'].fillna((limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\3146139932.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [49]:
df_mar_2017.shape

(5206, 16)

In [50]:
df_mar_2017

,customer_unique_id,customer_city,customer_state,review_score,price,freight_value,product_category_name_english,seller_city,seller_state,payment_type,payment_installments,payment_value,order_purchase_datetime,order_delivered_datetime,days_since_last_purchase,total_amount_client
55,e021e698833bdeb89dfef3acb2e91f37,jaragua do sul,SC,5.0,223.90,15.74,consoles_games,cascavel,PR,credit_card,4.0,239.64,2017-01-26 11:14:58,2017-02-09 12:15:20,582,239.64
90,cd076285a12f40041b32f5ad8c98699f,sao paulo,SP,4.0,14.99,15.56,drinks,pocos de caldas,MG,credit_card,4.0,61.10,2017-03-23 11:07:12,2017-04-04 14:58:07,526,61.10
98,d491a65a6ef3c04e145d37395996bad7,sao paulo,SP,5.0,18.99,8.72,telephony,sao paulo,SP,credit_card,1.0,27.71,2017-03-16 16:41:44,2017-03-21 11:27:49,533,27.71
122,90d479448b39ad9586d05da64cfb695b,sobral,CE,2.0,138.00,25.46,health_beauty,braganca paulista,SP,credit_card,2.0,163.46,2017-03-28 11:16:48,2017-05-17 15:41:52,521,163.46
132,4bc7250d4b42b3374eb6f41b12429592,sao paulo,SP,5.0,239.00,12.28,watches_gifts,sumare,SP,credit_card,8.0,251.28,2017-02-06 16:49:19,2017-02-11 12:38:56,571,251.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119101,5f2971f9805e3ccb030226e30c8e8390,sao paulo,SP,4.0,19.99,10.96,pet_shop,piracicaba,SP,credit_card,1.0,30.95,2017-03-27 23:04:18,2017-03-30 17:06:19,522,30.95
119118,1c137fe37df712015f6488edafe8ece4,aracitaba,MG,1.0,49.90,14.52,fashion_bags_accessories,sao paulo,SP,boleto,1.0,64.42,2017-01-25 16:51:27,2017-02-08 08:38:49,583,64.42
119128,874c93d867b18eb09a5e2f071ee89458,paulinia,SP,4.0,110.00,20.85,furniture_decor,santa rita do sapucai,MG,credit_card,1.0,130.85,2017-03-27 16:26:18,2017-04-04 13:57:53,522,130.85
119134,e7f8760e2bbd2f1986bebd99596c088e,belem,PA,3.0,59.90,24.42,stationery,sao paulo,SP,voucher,1.0,40.53,2017-03-15 21:04:08,2017-04-05 11:07:12,534,84.32


In [51]:
# Sélection des colonnes pertinentes pour la segmentation
selected_columns = ['total_amount_client', 'review_score', 
'days_since_last_purchase', 'payment_installments']

df_mar_2017 = df_mar_2017[selected_columns]

X = df_mar_2017[selected_columns]

# Prétraitement des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Application de l'algorithme K-means
kmeans_mar_2017 = KMeans(n_clusters = 4, random_state = 42)
kmeans_mar_2017.fit(X_scaled)

# Obtenir les labels prédits
kmeans_mar_2017_labels = kmeans_mar_2017.labels_

# Ajout des étiquettes de clusters au dataframe d'origine
df_mar_2017['cluster'] = kmeans_mar_2017.labels_

# Analyse des clusters
cluster_counts = df_mar_2017['cluster'].value_counts()
print("Nombre de clients par cluster:")
print(cluster_counts)

# drop de la colonne pour préserver la forme des données
# en vue d'un futur ARI
df_mar_2017 = df_mar_2017.drop(['cluster'], axis = 1)

Nombre de clients par cluster:
2    2046
0    1626
1     787
3     747
Name: cluster, dtype: int64


C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\3993707314.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mar_2017['cluster'] = kmeans_mar_2017.labels_


In [52]:
# Calculer la matrice de similarité entre les deux ensembles de clusters
similarity_matrix = pairwise_distances(kmeans_mar_2017_labels.reshape(-1, 1), 
kmeans_labels.reshape(-1, 1), metric='hamming')

# Calculer le Clustering Stability Index
stability_index = np.mean(similarity_matrix)

# Afficher le Clustering Stability Index
print("Clustering Stability Index:", stability_index)

Clustering Stability Index: 0.7113448127868481


In [53]:
adjusted_rand_score(kmeans_mar_2017.predict(df_mar_2017), 
kmeans.predict(df_mar_2017))

C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


0.9742591949469325

# <a name="C2">II. Simulations trimestrielles</a>

<font size="5">1. 1er Trimestre 2017</font>

In [54]:
#feature engineering

# Créer la variable du nombre de jours depuis le dernier achat
limit_date = pd.to_datetime('2018-09-01')
last_purchase = df_t1_2017.groupby('customer_unique_id')['order_purchase_datetime'].max()
df_t1_2017['days_since_last_purchase'] = (limit_date 
- df_t1_2017['customer_unique_id'].map(last_purchase)).dt.days

# remplacer les valeurs manquantes avec 
# le temps écoulé depuis la dernière observation
df_t1_2017['days_since_last_purchase'].fillna((limit_date 
- df_t1_2017['order_purchase_datetime']).dt.days, inplace = True)

# Créer la variable du montant total des achats effectués par le client
total_purchases = df_t1_2017.groupby('customer_unique_id')['payment_value'].sum()
df_t1_2017['total_amount_client'] = df_t1_2017['customer_unique_id'].map(total_purchases)

C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\4023110800.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t1_2017['days_since_last_purchase'] = (limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\4023110800.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t1_2017['days_since_last_purchase'].fillna((limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\4023110800.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [55]:
df_t1_2017.shape

(5206, 16)

In [56]:
df_t1_2017

,customer_unique_id,customer_city,customer_state,review_score,price,freight_value,product_category_name_english,seller_city,seller_state,payment_type,payment_installments,payment_value,order_purchase_datetime,order_delivered_datetime,days_since_last_purchase,total_amount_client
55,e021e698833bdeb89dfef3acb2e91f37,jaragua do sul,SC,5.0,223.90,15.74,consoles_games,cascavel,PR,credit_card,4.0,239.64,2017-01-26 11:14:58,2017-02-09 12:15:20,582,239.64
90,cd076285a12f40041b32f5ad8c98699f,sao paulo,SP,4.0,14.99,15.56,drinks,pocos de caldas,MG,credit_card,4.0,61.10,2017-03-23 11:07:12,2017-04-04 14:58:07,526,61.10
98,d491a65a6ef3c04e145d37395996bad7,sao paulo,SP,5.0,18.99,8.72,telephony,sao paulo,SP,credit_card,1.0,27.71,2017-03-16 16:41:44,2017-03-21 11:27:49,533,27.71
122,90d479448b39ad9586d05da64cfb695b,sobral,CE,2.0,138.00,25.46,health_beauty,braganca paulista,SP,credit_card,2.0,163.46,2017-03-28 11:16:48,2017-05-17 15:41:52,521,163.46
132,4bc7250d4b42b3374eb6f41b12429592,sao paulo,SP,5.0,239.00,12.28,watches_gifts,sumare,SP,credit_card,8.0,251.28,2017-02-06 16:49:19,2017-02-11 12:38:56,571,251.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119101,5f2971f9805e3ccb030226e30c8e8390,sao paulo,SP,4.0,19.99,10.96,pet_shop,piracicaba,SP,credit_card,1.0,30.95,2017-03-27 23:04:18,2017-03-30 17:06:19,522,30.95
119118,1c137fe37df712015f6488edafe8ece4,aracitaba,MG,1.0,49.90,14.52,fashion_bags_accessories,sao paulo,SP,boleto,1.0,64.42,2017-01-25 16:51:27,2017-02-08 08:38:49,583,64.42
119128,874c93d867b18eb09a5e2f071ee89458,paulinia,SP,4.0,110.00,20.85,furniture_decor,santa rita do sapucai,MG,credit_card,1.0,130.85,2017-03-27 16:26:18,2017-04-04 13:57:53,522,130.85
119134,e7f8760e2bbd2f1986bebd99596c088e,belem,PA,3.0,59.90,24.42,stationery,sao paulo,SP,voucher,1.0,40.53,2017-03-15 21:04:08,2017-04-05 11:07:12,534,84.32


In [57]:
# Sélection des colonnes pertinentes pour la segmentation
selected_columns = ['total_amount_client', 'review_score', 
'days_since_last_purchase', 'payment_installments']

df_t1_2017 = df_t1_2017[selected_columns]

X = df_t1_2017[selected_columns]

# Prétraitement des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Application de l'algorithme K-means
kmeans_t1_2017 = KMeans(n_clusters = 4, random_state = 42)
kmeans_t1_2017.fit(X_scaled)

# Obtenir les labels prédits
kmeans_t1_2017_labels = kmeans_t1_2017.labels_

# Ajout des étiquettes de clusters au dataframe d'origine
df_t1_2017['cluster'] = kmeans_t1_2017.labels_

# Analyse des clusters
cluster_counts = df_t1_2017['cluster'].value_counts()
print("Nombre de clients par cluster:")
print(cluster_counts)

# drop de la colonne pour préserver la forme des données
# en vue d'un futur ARI
df_t1_2017 = df_t1_2017.drop(['cluster'], axis = 1)

Nombre de clients par cluster:
2    2046
0    1626
1     787
3     747
Name: cluster, dtype: int64


C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\319529868.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t1_2017['cluster'] = kmeans_t1_2017.labels_


In [58]:
# Calculer la matrice de similarité entre les deux ensembles de clusters
similarity_matrix = pairwise_distances(kmeans_t1_2017_labels.reshape(-1, 1), 
kmeans_labels.reshape(-1, 1), metric='hamming')

# Calculer le Clustering Stability Index
stability_index = np.mean(similarity_matrix)

# Afficher le Clustering Stability Index
print("Clustering Stability Index:", stability_index)

Clustering Stability Index: 0.7113448127868481


In [59]:
adjusted_rand_score(kmeans_t1_2017.predict(df_t1_2017), 
kmeans.predict(df_t1_2017))

C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


0.9742591949469325

<font size="5">2. 2eme Trimestre 2017</font>

In [60]:
#feature engineering

# Créer la variable du nombre de jours depuis le dernier achat
limit_date = pd.to_datetime('2018-09-01')
last_purchase = df_t2_2017.groupby('customer_unique_id')['order_purchase_datetime'].max()
df_t2_2017['days_since_last_purchase'] = (limit_date 
- df_t2_2017['customer_unique_id'].map(last_purchase)).dt.days

# remplacer les valeurs manquantes avec 
# le temps écoulé depuis la dernière observation
df_t2_2017['days_since_last_purchase'].fillna((limit_date 
- df_t2_2017['order_purchase_datetime']).dt.days, inplace = True)

# Créer la variable du montant total des achats effectués par le client
total_purchases = df_t2_2017.groupby('customer_unique_id')['payment_value'].sum()
df_t2_2017['total_amount_client'] = df_t2_2017['customer_unique_id'].map(total_purchases)

C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\2465848322.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t2_2017['days_since_last_purchase'] = (limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\2465848322.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t2_2017['days_since_last_purchase'].fillna((limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\2465848322.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [61]:
df_t2_2017.shape

(9507, 16)

In [62]:
df_t2_2017

,customer_unique_id,customer_city,customer_state,review_score,price,freight_value,product_category_name_english,seller_city,seller_state,payment_type,payment_installments,payment_value,order_purchase_datetime,order_delivered_datetime,days_since_last_purchase,total_amount_client
0,861eff4711a542e4b93843c6dd7febb0,franca,SP,4.0,124.99,21.88,office_furniture,itaquaquecetuba,SP,credit_card,2.0,146.87,2017-05-16 15:05:35,2017-05-25 10:35:35,472,146.87
26,4390ddbb6276a66ff1736a6710205dca,curitiba,PR,5.0,79.90,14.73,baby,maua,SP,credit_card,1.0,94.63,2017-04-14 11:24:56,2017-04-20 17:04:51,504,94.63
30,2e6a42a9b5cbb0da62988694f18ee295,sao paulo,SP,3.0,29.99,17.60,kitchen_dining_laundry_garden_furniture,toledo,PR,credit_card,1.0,47.59,2017-05-15 17:58:25,2017-05-19 11:10:11,473,47.59
36,e079b18794454de9d2be5c12b4392294,resende,RJ,5.0,56.99,16.13,bed_bath_table,ibitinga,SP,credit_card,8.0,134.25,2017-06-14 18:31:54,2017-07-07 20:32:47,443,1482.62
37,e079b18794454de9d2be5c12b4392294,resende,RJ,5.0,44.99,16.14,bed_bath_table,ibitinga,SP,credit_card,8.0,134.25,2017-06-14 18:31:54,2017-07-07 20:32:47,443,1482.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119085,db634985638ec4b6abf8734d7adde1ab,curitiba,PR,5.0,129.00,15.66,toys,santo andre,SP,credit_card,3.0,144.66,2017-06-23 10:55:02,2017-07-03 17:14:56,434,144.66
119089,a2325193ac2e95b805642e3100f80af3,ilhabela,SP,5.0,199.00,20.63,cool_stuff,montenegro,RS,credit_card,4.0,219.63,2017-06-12 10:34:29,2017-06-22 16:36:36,445,219.63
119117,c811311629fd3d76f5ca5eeb2abfa7e8,marilia,SP,4.0,47.99,10.96,fashion_bags_accessories,sao paulo,SP,credit_card,1.0,58.95,2017-04-10 00:08:56,2017-04-15 08:13:25,508,58.95
119122,206e64e8af2633a2ebe158a7fcb860db,poa,SP,1.0,89.90,12.13,home_confort,ibitinga,SP,boleto,1.0,102.03,2017-05-15 17:42:38,2017-05-22 14:14:45,473,102.03


In [63]:
# Sélection des colonnes pertinentes pour la segmentation
selected_columns = ['total_amount_client', 'review_score', 
'days_since_last_purchase', 'payment_installments']

df_t2_2017 = df_t2_2017[selected_columns]

X = df_t2_2017[selected_columns]

# Prétraitement des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Application de l'algorithme K-means
kmeans_t2_2017 = KMeans(n_clusters = 4, random_state = 42)
kmeans_t2_2017.fit(X_scaled)

# Obtenir les labels prédits
kmeans_t2_2017_labels = kmeans_t2_2017.labels_

# Ajout des étiquettes de clusters au dataframe d'origine
df_t2_2017['cluster'] = kmeans_t2_2017.labels_

# Analyse des clusters
cluster_counts = df_t2_2017['cluster'].value_counts()
print("Nombre de clients par cluster:")
print(cluster_counts)

# drop de la colonne pour préserver la forme des données
# en vue d'un futur ARI
df_t2_2017 = df_t2_2017.drop(['cluster'], axis = 1)

Nombre de clients par cluster:
0    3549
2    3125
3    1463
1    1370
Name: cluster, dtype: int64


C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\3535615005.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t2_2017['cluster'] = kmeans_t2_2017.labels_


In [64]:
# Calculer la matrice de similarité entre les deux ensembles de clusters
similarity_matrix = pairwise_distances(kmeans_t2_2017_labels.reshape(-1, 1), 
kmeans_labels.reshape(-1, 1), metric='hamming')

# Calculer le Clustering Stability Index
stability_index = np.mean(similarity_matrix)

# Afficher le Clustering Stability Index
print("Clustering Stability Index:", stability_index)

Clustering Stability Index: 0.7052962500198896


In [65]:
adjusted_rand_score(kmeans_t2_2017.predict(df_t2_2017), 
kmeans.predict(df_t2_2017))

C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


0.9903662625678834

<font size="5">3. 3eme Trimestre 2017</font>

In [66]:
#feature engineering

# Créer la variable du nombre de jours depuis le dernier achat
limit_date = pd.to_datetime('2018-09-01')
last_purchase = df_t3_2017.groupby('customer_unique_id')['order_purchase_datetime'].max()
df_t3_2017['days_since_last_purchase'] = (limit_date 
- df_t3_2017['customer_unique_id'].map(last_purchase)).dt.days

# remplacer les valeurs manquantes avec 
# le temps écoulé depuis la dernière observation
df_t3_2017['days_since_last_purchase'].fillna((limit_date 
- df_t3_2017['order_purchase_datetime']).dt.days, inplace = True)

# Créer la variable du montant total des achats effectués par le client
total_purchases = df_t3_2017.groupby('customer_unique_id')['payment_value'].sum()
df_t3_2017['total_amount_client'] = df_t3_2017['customer_unique_id'].map(total_purchases)

C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\2500611010.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t3_2017['days_since_last_purchase'] = (limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\2500611010.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t3_2017['days_since_last_purchase'].fillna((limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\2500611010.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [67]:
df_t3_2017.shape

(12949, 16)

In [68]:
df_t3_2017

,customer_unique_id,customer_city,customer_state,review_score,price,freight_value,product_category_name_english,seller_city,seller_state,payment_type,payment_installments,payment_value,order_purchase_datetime,order_delivered_datetime,days_since_last_purchase,total_amount_client
5,4c93744516667ad3b8f1fb645a3116a4,jaragua do sul,SC,5.0,259.90,22.31,sports_leisure,pirituba,SP,debit_card,1.0,282.21,2017-09-14 18:14:31,2017-09-28 17:32:43,351,282.21
12,918dc87cd72cd9f6ed4bd442ed785235,lencois paulista,SP,4.0,99.00,18.31,air_conditioning,jacutinga,MG,credit_card,1.0,117.31,2017-09-09 09:54:57,2017-09-20 20:23:34,356,117.31
21,e607ede0e63436308660236f5a52da5e,florianopolis,SC,5.0,105.00,18.00,computers_accessories,porto alegre,RS,credit_card,1.0,123.00,2017-08-13 10:03:36,2017-08-19 12:37:49,383,123.00
33,424aca6872c5bab80780a8dec03b7516,sao jose dos campos,SP,4.0,59.90,13.44,garden_tools,sao jose do rio preto,SP,boleto,1.0,73.34,2017-09-12 08:00:03,2017-09-25 21:22:39,353,73.34
42,40febde16f4718a5def537786473b0be,novo hamburgo,RS,5.0,44.90,15.10,art,blumenau,SC,credit_card,2.0,60.00,2017-08-10 13:01:54,2017-08-21 13:42:55,386,60.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119059,8c8173e547e020f411aa55b2fceed861,paicandu,PR,5.0,59.90,17.67,auto,penapolis,SP,boleto,1.0,77.57,2017-08-01 12:57:02,2017-08-15 13:54:33,395,77.57
119064,a10c0d1e68e919e3d91ed940a0ba6aed,rio de janeiro,RJ,2.0,55.00,7.94,watches_gifts,sao goncalo,RJ,credit_card,5.0,125.88,2017-09-18 10:42:04,2017-10-05 17:48:03,347,125.88
119068,a5ba329297100ea689fa263768b35b8b,caxias do sul,RS,3.0,35.00,5.37,housewares,sao paulo,SP,credit_card,10.0,115.30,2017-07-15 14:31:13,2017-07-28 18:32:36,412,230.60
119069,a5ba329297100ea689fa263768b35b8b,caxias do sul,RS,3.0,15.99,21.48,furniture_decor,assis,SP,credit_card,10.0,115.30,2017-07-15 14:31:13,2017-07-28 18:32:36,412,230.60


In [69]:
# Sélection des colonnes pertinentes pour la segmentation
selected_columns = ['total_amount_client', 'review_score', 
'days_since_last_purchase', 'payment_installments']

df_t3_2017 = df_t3_2017[selected_columns]

X = df_t3_2017[selected_columns]

# Prétraitement des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Application de l'algorithme K-means
kmeans_t3_2017 = KMeans(n_clusters = 4, random_state = 42)
kmeans_t3_2017.fit(X_scaled)

# Obtenir les labels prédits
kmeans_t3_2017_labels = kmeans_t3_2017.labels_

# Ajout des étiquettes de clusters au dataframe d'origine
df_t3_2017['cluster'] = kmeans_t3_2017.labels_

# Analyse des clusters
cluster_counts = df_t3_2017['cluster'].value_counts()
print("Nombre de clients par cluster:")
print(cluster_counts)

# drop de la colonne pour préserver la forme des données
# en vue d'un futur ARI
df_t3_2017 = df_t3_2017.drop(['cluster'], axis = 1)

Nombre de clients par cluster:
0    4757
2    4702
1    1859
3    1631
Name: cluster, dtype: int64


C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\295944354.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t3_2017['cluster'] = kmeans_t3_2017.labels_


In [70]:
# Calculer la matrice de similarité entre les deux ensembles de clusters
similarity_matrix = pairwise_distances(kmeans_t3_2017_labels.reshape(-1, 1), 
kmeans_labels.reshape(-1, 1), metric='hamming')

# Calculer le Clustering Stability Index
stability_index = np.mean(similarity_matrix)

# Afficher le Clustering Stability Index
print("Clustering Stability Index:", stability_index)

Clustering Stability Index: 0.7017754301127173


In [71]:
adjusted_rand_score(kmeans_t3_2017.predict(df_t3_2017), 
kmeans.predict(df_t3_2017))

C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


0.9020688894958007

<font size="5">4. 4eme Trimestre 2017</font>

In [72]:
#feature engineering

# Créer la variable du nombre de jours depuis le dernier achat
limit_date = pd.to_datetime('2018-09-01')
last_purchase = df_t4_2017.groupby('customer_unique_id')['order_purchase_datetime'].max()
df_t4_2017['days_since_last_purchase'] = (limit_date 
- df_t4_2017['customer_unique_id'].map(last_purchase)).dt.days

# remplacer les valeurs manquantes avec 
# le temps écoulé depuis la dernière observation
df_t4_2017['days_since_last_purchase'].fillna((limit_date 
- df_t4_2017['order_purchase_datetime']).dt.days, inplace = True)

# Créer la variable du montant total des achats effectués par le client
total_purchases = df_t4_2017.groupby('customer_unique_id')['payment_value'].sum()
df_t4_2017['total_amount_client'] = df_t4_2017['customer_unique_id'].map(total_purchases)

C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\297136808.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t4_2017['days_since_last_purchase'] = (limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\297136808.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t4_2017['days_since_last_purchase'].fillna((limit_date
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\297136808.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [73]:
df_t4_2017.shape

(18116, 16)

In [74]:
df_t4_2017

,customer_unique_id,customer_city,customer_state,review_score,price,freight_value,product_category_name_english,seller_city,seller_state,payment_type,payment_installments,payment_value,order_purchase_datetime,order_delivered_datetime,days_since_last_purchase,total_amount_client
10,2a7745e1ed516b289ed9b29c7d0539a5,montes claros,MG,5.0,25.30,15.10,auto,piracicaba,SP,credit_card,1.0,40.40,2017-11-27 17:23:20,2017-12-08 22:16:17,277,40.40
20,3e6fd6b2f0d499456a6a6820a40f2d79,pacaja,PA,4.0,28.99,25.63,telephony,sao paulo,SP,credit_card,5.0,54.62,2017-11-01 21:54:10,2017-12-02 01:09:29,303,54.62
22,a96d5cfa0d3181817e2b946f921ea021,aparecida de goiania,GO,1.0,59.90,17.67,garden_tools,sao jose do rio preto,SP,credit_card,2.0,232.71,2017-10-15 11:08:48,2017-10-25 22:30:58,320,232.71
31,4d221875624017bc47b4d1ce7314a5b7,cachoeiro de itapemirim,ES,2.0,53.99,15.13,perfumery,santo andre,SP,credit_card,6.0,69.12,2017-11-24 11:29:52,2017-12-06 22:37:09,280,69.12
34,bf4862777db128507e9efcc789215e9b,sao roque,SP,5.0,49.00,13.44,garden_tools,sao jose do rio preto,SP,boleto,1.0,62.44,2017-11-26 20:59:24,2017-12-09 11:56:34,278,62.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119129,82d46759af0369aad49084bacf85a6c3,bom repouso,MG,1.0,167.99,31.93,electronics,braganca paulista,SP,credit_card,5.0,167.79,2017-11-14 12:04:09,2017-12-19 19:37:33,290,199.92
119130,82d46759af0369aad49084bacf85a6c3,bom repouso,MG,1.0,167.99,31.93,electronics,braganca paulista,SP,credit_card,1.0,32.13,2017-11-14 12:04:09,2017-12-19 19:37:33,290,199.92
119137,4b5820135d360a45552b5163835b1d89,divinopolis,MG,1.0,55.00,20.88,housewares,sao paulo,SP,credit_card,1.0,75.88,2017-12-17 23:13:41,2018-01-04 13:08:05,257,75.88
119141,73c2643a0a458b49f58cea58833b192e,canoas,RS,5.0,689.00,22.07,watches_gifts,guariba,SP,credit_card,2.0,711.07,2017-11-03 21:08:33,2017-11-16 19:58:39,301,711.07


In [75]:
# Sélection des colonnes pertinentes pour la segmentation
selected_columns = ['total_amount_client', 'review_score', 
'days_since_last_purchase', 'payment_installments']

df_t4_2017 = df_t4_2017[selected_columns]

X = df_t4_2017[selected_columns]

# Prétraitement des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Application de l'algorithme K-means
kmeans_t4_2017 = KMeans(n_clusters = 4, random_state = 42)
kmeans_t4_2017.fit(X_scaled)

# Obtenir les labels prédits
kmeans_t4_2017_labels = kmeans_t4_2017.labels_

# Ajout des étiquettes de clusters au dataframe d'origine
df_t4_2017['cluster'] = kmeans_t4_2017.labels_

# Analyse des clusters
cluster_counts = df_t4_2017['cluster'].value_counts()
print("Nombre de clients par cluster:")
print(cluster_counts)

# drop de la colonne pour préserver la forme des données
# en vue d'un futur ARI
df_t4_2017 = df_t4_2017.drop(['cluster'], axis = 1)

Nombre de clients par cluster:
1    7751
0    4852
2    3109
3    2404
Name: cluster, dtype: int64


C:\Users\Oliver\AppData\Local\Temp\ipykernel_19704\2706579546.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t4_2017['cluster'] = kmeans_t4_2017.labels_


In [76]:
# Calculer la matrice de similarité entre les deux ensembles de clusters
similarity_matrix = pairwise_distances(kmeans_t4_2017_labels.reshape(-1, 1), 
kmeans_labels.reshape(-1, 1), metric='hamming')

# Calculer le Clustering Stability Index
stability_index = np.mean(similarity_matrix)

# Afficher le Clustering Stability Index
print("Clustering Stability Index:", stability_index)

Clustering Stability Index: 0.7612116045608548


In [77]:
adjusted_rand_score(kmeans_t4_2017.predict(df_t4_2017), 
kmeans.predict(df_t4_2017))

C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
C:\Users\Oliver\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


0.8012904187604583